In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
#   Convert RGB images to CIE-LAB Color File

from __future__ import print_function
import argparse
import os

import numpy as np
import pickle
import random
from keras.preprocessing.image import list_pictures, array_to_img
from image_ext import list_pictures_in_multidir, load_imgs_asarray, img_dice_coeff
from fname_func import make_fnames, load_fnames

np.random.seed(2016)

Using TensorFlow backend.


In [2]:
def RGB2RGChroma(img):
    # convert RGB to RG Chromaticity Image
    # https://en.wikipedia.org/wiki/Rg_chromaticity
    ret = np.zeros((img.shape[0],img.shape[1],img.shape[2]),'float')
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    I = R + G + B + 1
    r = (R+1)/I
    g = (G+1)/I
    b = (B+1)/I
    R2 = r*G / g
    B2 = ((1 - r - g)*G) / g
    
    ret[:,:,0] = r
    ret[:,:,1] = g
    ret[:,:,2] = 0
    
    return ret

In [3]:
def LabConversion(img):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2LAB)
    return ret

In [4]:
def LabAugumentation(img,L,a,b):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_BGR2LAB)
    # print(ret)
    ret[:,:,0] = ret[:,:,0] + L
    ret[:,:,1] = ret[:,:,1] + a
    ret[:,:,2] = ret[:,:,2] + b
    ret = np.clip(ret,0,255)
    ret = cv2.cvtColor(ret, cv2.COLOR_LAB2BGR)    
    return ret

In [8]:
def ReflectionAugumentation(im, im_irismask, im_reflection, alpha):
    tmp = cv2.resize(im_reflection, (im.shape[1],im.shape[0]))
    mask = cv2.resize(im_irismask, (im.shape[1],im.shape[0]))
    mask[mask > 0] = 1
    im2 = im.astype(np.float64) + tmp.astype(np.float64)*mask.astype(np.float64)*alpha
    im2[im2 > 255] = 255
    return im2.astype(np.uint8)

In [12]:
#
#  MAIN STARTS FROM HERE
#
if __name__ == '__main__':
    
    target_size = (224, 224)
    dim_ordering = 'channels_last'
    
#    fnames = load_fnames('data.nnlab/list_files_all.txt')
#    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data.nnlab/image','data.nnlab/gt','')
#    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data.nnlab.LAB/img','data.nnlab/gt','')
    fnames = load_fnames('data/list_files_all.txt')
    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data/img','data/mask','OperatorA_')
    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data_augumented_2/img','data_augumented_2/mask','OperatorA_')

    if os.path.exists('data_augumented_2') == 0:
        os.mkdir('data_augumented_2')
        os.mkdir('data_augumented_2/img')
        os.mkdir('data_augumented_2/mask')
    
    simulate_reflection = 1
    MAX_ALPHA = 0.2
    
    if simulate_reflection == 1:
        fnames_sc = load_fnames('data.scenes/list_files.txt')
        [fpaths_sc,tmp] = make_fnames(fnames_sc,'data.scenes/','data.scenes/','')
        [tmp,fpaths_irismask] = make_fnames(fnames,'data/img','data/mask_iris','OperatorA_') 
        num_reflection = len(fnames_sc)

In [13]:
    import matplotlib.pyplot as plt
    from PIL import Image 
    import cv2
    
    aug_div = [20,30,30]    
    
    fpath_img = 'data_augumented_2/img'
    fpath_mask = 'data_augumented_2/mask'

In [15]:
    for i in range(len(fpaths_xs_train)):
        img = cv2.imread(fpaths_xs_train[i])
        mask = cv2.imread(fpaths_ys_train[i])
        
        fnames_img = fpath_img + '/' + fnames[i];
        cv2.imwrite(fnames_img, img)
        
        fnames_mask = fpath_mask + '/' + fnames[i];
        print(fnames_mask)
        cv2.imwrite(fnames_mask,mask)
        
        im_irismask = cv2.imread(fpaths_irismask[i])
        
        for j in range(3):
            aug_param = [ random.randint(-aug_div[0],aug_div[0]), random.randint(-aug_div[1],aug_div[1]),
                             random.randint(-aug_div[2],aug_div[2]) ]
            print(aug_param)
            im = LabAugumentation(img,aug_param[0],aug_param[1],aug_param[2])
            
            if simulate_reflection == 1:
                im_reflection = cv2.imread(fpaths_sc[random.randint(0,num_reflection)])

                alpha = random.uniform(0,0.5)
                im2 = ReflectionAugumentation(im, im_irismask, im_reflection, alpha)
            
            fnames_img = '%s/%03d_%s'%(fpath_img, j, fnames[i])
            fnames_mask = '%s/%03d_%s'%(fpath_mask, j, fnames[i])
            print(fnames_img)
            print(fnames_mask)
            cv2.imwrite(fnames_img, im2)
            cv2.imwrite(fnames_mask, mask)
        

data_augumented_2/mask/C100_S1_I1.tiff
[20, 0, 3]
data_augumented_2/img/000_C100_S1_I1.tiff
data_augumented_2/mask/000_C100_S1_I1.tiff
[3, 5, 0]
data_augumented_2/img/001_C100_S1_I1.tiff
data_augumented_2/mask/001_C100_S1_I1.tiff
[16, -1, -29]
data_augumented_2/img/002_C100_S1_I1.tiff
data_augumented_2/mask/002_C100_S1_I1.tiff
data_augumented_2/mask/C100_S1_I10.tiff
[6, 20, 21]
data_augumented_2/img/000_C100_S1_I10.tiff
data_augumented_2/mask/000_C100_S1_I10.tiff
[0, -8, -17]
data_augumented_2/img/001_C100_S1_I10.tiff
data_augumented_2/mask/001_C100_S1_I10.tiff
[10, 5, -28]
data_augumented_2/img/002_C100_S1_I10.tiff
data_augumented_2/mask/002_C100_S1_I10.tiff
data_augumented_2/mask/C100_S1_I11.tiff
[8, -9, 17]
data_augumented_2/img/000_C100_S1_I11.tiff
data_augumented_2/mask/000_C100_S1_I11.tiff
[4, 15, -9]
data_augumented_2/img/001_C100_S1_I11.tiff
data_augumented_2/mask/001_C100_S1_I11.tiff
[-11, 5, -3]
data_augumented_2/img/002_C100_S1_I11.tiff
data_augumented_2/mask/002_C100_S1_I11

data_augumented_2/img/001_C100_S2_I4.tiff
data_augumented_2/mask/001_C100_S2_I4.tiff
[17, -1, -27]
data_augumented_2/img/002_C100_S2_I4.tiff
data_augumented_2/mask/002_C100_S2_I4.tiff
data_augumented_2/mask/C100_S2_I5.tiff
[-1, -12, -10]
data_augumented_2/img/000_C100_S2_I5.tiff
data_augumented_2/mask/000_C100_S2_I5.tiff
[4, -5, -16]
data_augumented_2/img/001_C100_S2_I5.tiff
data_augumented_2/mask/001_C100_S2_I5.tiff
[2, 18, -13]
data_augumented_2/img/002_C100_S2_I5.tiff
data_augumented_2/mask/002_C100_S2_I5.tiff
data_augumented_2/mask/C100_S2_I6.tiff
[16, -8, -19]
data_augumented_2/img/000_C100_S2_I6.tiff
data_augumented_2/mask/000_C100_S2_I6.tiff
[-9, -9, 6]
data_augumented_2/img/001_C100_S2_I6.tiff
data_augumented_2/mask/001_C100_S2_I6.tiff
[-1, -15, -30]
data_augumented_2/img/002_C100_S2_I6.tiff
data_augumented_2/mask/002_C100_S2_I6.tiff
data_augumented_2/mask/C100_S2_I7.tiff
[18, -15, -22]
data_augumented_2/img/000_C100_S2_I7.tiff
data_augumented_2/mask/000_C100_S2_I7.tiff
[7, -30

data_augumented_2/mask/C101_S2_I13.tiff
[-17, 19, 9]
data_augumented_2/img/000_C101_S2_I13.tiff
data_augumented_2/mask/000_C101_S2_I13.tiff
[18, -18, -15]
data_augumented_2/img/001_C101_S2_I13.tiff
data_augumented_2/mask/001_C101_S2_I13.tiff
[-7, -27, 30]
data_augumented_2/img/002_C101_S2_I13.tiff
data_augumented_2/mask/002_C101_S2_I13.tiff
data_augumented_2/mask/C101_S2_I14.tiff
[-6, -27, 6]
data_augumented_2/img/000_C101_S2_I14.tiff
data_augumented_2/mask/000_C101_S2_I14.tiff
[16, 18, 0]
data_augumented_2/img/001_C101_S2_I14.tiff
data_augumented_2/mask/001_C101_S2_I14.tiff
[5, 16, 24]
data_augumented_2/img/002_C101_S2_I14.tiff
data_augumented_2/mask/002_C101_S2_I14.tiff
data_augumented_2/mask/C101_S2_I15.tiff
[-10, 9, -15]
data_augumented_2/img/000_C101_S2_I15.tiff
data_augumented_2/mask/000_C101_S2_I15.tiff
[-10, 21, 9]
data_augumented_2/img/001_C101_S2_I15.tiff
data_augumented_2/mask/001_C101_S2_I15.tiff
[-4, 12, 19]
data_augumented_2/img/002_C101_S2_I15.tiff
data_augumented_2/mask

IndexError: list index out of range